## Nodes_by_areas_links_by_modes

### Import libraries

In [1]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
from sqlalchemy import create_engine
from sqlalchemy import text
from shapely.geometry import LineString, Point
from shapely.ops import unary_union

#### Create the connection with the DB

In [2]:
# Step 1: Create the connection with the DB
engine = create_engine("postgresql://urbaninfo:@cirrus.ita.chalmers.se/se_tuptp")
conn = engine.connect()

In [3]:
# Step 2: Create a new schema when it is necessary
#schema = '''CREATE SCHEMA name_schema'''
#conn.execute(schema)

### Nodes

In [4]:
# Import Data
# Step 0: Import Data
nodes_pt_1_query = text('SELECT * FROM pt_1_mdirections_mstops_mroutes.nodes_pt_1')
nodes_pt_1 = gpd.read_postgis(nodes_pt_1_query, engine, geom_col='geometry')

In [5]:
# Preprocess the data
# Step 1: Copy the original DataFrame
nodes_pt_5_l1 = nodes_pt_1.copy()

# Step 2: Aggregate functions for node grouping
agg_func_node = {
    'route_id': list,
    'route_short_name': list,
    'stop_id': list,
    'direction_id': list,
    'mode': list,
    'osmid': list,
    'geometry': lambda g: Point(unary_union(g).centroid.x, unary_union(g).centroid.y)
}
nodes_pt_5_l2 = nodes_pt_5_l1.groupby(['stop_name', 'place_id']).agg(agg_func_node).reset_index()

# Step 3: Set Geometry
nodes_pt_5_l2 = gpd.GeoDataFrame(nodes_pt_5_l2, geometry='geometry', crs='EPSG:3006')

# Step 4: Display the results
nodes_pt_5_l2.head()

,stop_name,place_id,route_id,route_short_name,stop_id,direction_id,mode,osmid,geometry
0,7:e Villagatan,307955489,"[9011014200300000, 9011014200500000, 901101420...","[3, 5, 8, 1, 3, 5, 8, 1]","[9022014082785001, 9022014082785001, 902201408...","[0, 1, 0, 0, 1, 0, 1, 1]","[bus_service, bus_service, bus_service, bus_se...","[87793055.0, 87793055.0, 87793055.0, 87793055....",POINT (378291.977 6399668.440)
1,AGA Vattenfall,307980814,"[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 932, 331, 334, 4]","[9022014015161001, 9022014015161001, 902201401...","[0, 0, 1, 1, 1, 1, 0]","[bus_service, bus_service, bus_service, bus_se...","[1021141732.0, 1021141732.0, 1021141732.0, 102...",POINT (313191.955 6442732.480)
2,Abrahamstorp,307959538,"[9011014366300000, 9011014658100000, 901101436...","[663, 581, 663]","[9022014046020001, 9022014046020001, 902201404...","[1, 1, 0]","[communal_taxi_service, bus_service, communal_...","[173516827.0, 173516827.0, 835571452.0]",POINT (371017.481 6448504.455)
3,Acklinga,307927983,"[9011014330300000, 9011014332700000, 901101438...","[303, 327, 327, 303, 327]","[9022014042120001, 9022014042120001, 902201404...","[1, 1, 0, 0, 0]","[bus_service, bus_service, communal_taxi_servi...","[19804825.0, 19804825.0, 19804825.0, 19804825....",POINT (434427.979 6451013.415)
4,Acklinga Solbacken,307927983,"[9011014384000000, 9011014332600000]","[326, 326]","[9022014042270001, 9022014042270002]","[1, 1]","[communal_taxi_service, bus_service]","[1008032531.0, 1008032531.0]",POINT (434327.469 6451981.944)


In [6]:
# Step 5: Create buffer zones
nodes_pt_5_l2 ['buffer'] = nodes_pt_5_l2.geometry.buffer(85)
nodes_pt_5_l2 = nodes_pt_5_l2.set_geometry('buffer', crs='epsg:3006')

In [7]:
# Step 6: Create unary union GeoDataFrame
nodes_pt_5_l2_unary_union_buffer = gpd.GeoDataFrame(
    geometry=[nodes_pt_5_l2.unary_union], crs='epsg:3006'
).explode(index_parts=True).reset_index(drop=True)

# Step 7: Assign buffer IDs
nodes_pt_5_l2_unary_union_buffer['buffer_id'] = nodes_pt_5_l2_unary_union_buffer.index

In [8]:
# Step 8: Perform spatial join
nodes_pt_5_l2_spatial_join = (
    gpd.sjoin(nodes_pt_5_l2_unary_union_buffer, nodes_pt_5_l2, how='inner', predicate='intersects')
    .reset_index(drop=True)
    .rename(columns={'geometry_left': 'geometry_buffer', 'geometry_right': 'geometry'})
)

In [9]:
# Step 9: Explode modes
nodes_pt_5_l2_exploded_modes = nodes_pt_5_l2_spatial_join.explode('mode').drop_duplicates(['stop_name', 'place_id', 'mode'])

# Step 10: Count modes by buffer and merge the mode count in the nodes table
nodes_pt_5_l2_mode_count = nodes_pt_5_l2_exploded_modes.groupby('buffer_id')['mode'].nunique().reset_index(name='mode_count')
nodes_pt_5_l2_mode_count_merged = pd.merge(nodes_pt_5_l2_exploded_modes, nodes_pt_5_l2_mode_count, on='buffer_id', how='inner')

In [10]:
# Step 11: Create tables with stops having more than one mode and with only one mode
nodes_pt_5_l2_multi_modes = nodes_pt_5_l2_mode_count_merged[nodes_pt_5_l2_mode_count_merged.mode_count > 1]
nodes_pt_5_l2_one_mode = nodes_pt_5_l2_mode_count_merged[nodes_pt_5_l2_mode_count_merged.mode_count == 1]

In [11]:
# Step 12: Rename buffers by most recurrent stop name
nodes_pt_5_l2_stop_areas_name = nodes_pt_5_l2_multi_modes.groupby('buffer_id').agg({'stop_name': lambda x: x.mode().iloc[0]}).reset_index()

In [12]:
# Step 13: Merge new names with multi-mode buffers
nodes_pt_5_l2_stop_areas_name_multi_modes = pd.merge(
    nodes_pt_5_l2_multi_modes[['buffer_id', 'stop_name', 'place_id', 'mode', 'geometry_buffer']],
    nodes_pt_5_l2_stop_areas_name, on='buffer_id', how='inner'
).rename(columns={'stop_name_x': 'stop_name', 'stop_name_y': 'stop_areas_name'})

In [13]:
# Step 14: Create centroids of buffers
nodes_pt_5_l2_stop_areas_centroids = gpd.GeoDataFrame(nodes_pt_5_l2_stop_areas_name_multi_modes, geometry='geometry_buffer', crs='EPSG:3006')
nodes_pt_5_l2_stop_areas_centroids ['geometry'] = nodes_pt_5_l2_stop_areas_centroids.geometry.centroid

In [14]:
nodes_pt_5_l2_one_mode

,geometry_buffer,buffer_id,index_right,stop_name,place_id,route_id,route_short_name,stop_id,direction_id,mode,osmid,geometry,mode_count
0,"POLYGON ((285973.651 6446939.654, 285965.560 6...",0,4097,Käringön,307987417,[9011014638100000],[381],[9022014015746001],[0],water_transport_service,[306120486.0],POINT (285948.977 6447020.994),1
1,"POLYGON ((284942.625 6470902.629, 284934.534 6...",1,1339,Draget,307856454,[9011014436600000],[866],[9022014024333001],[0],communal_taxi_service,[644790439.0],POINT (284917.951 6470983.969),1
2,"POLYGON ((285037.134 6472412.116, 285029.043 6...",2,5433,Paradiset,307856454,"[9011014436600000, 9011014436600000]","[866, 866]","[9022014024326001, 9022014024326002]","[1, 0]",communal_taxi_service,"[644771938.0, 644771938.0]",POINT (285012.460 6472493.456),1
3,"POLYGON ((285312.667 6472742.149, 285304.576 6...",3,3955,Kursgården,307856454,"[9011014436600000, 9011014436600000]","[866, 866]","[9022014024334001, 9022014024334002]","[1, 0]",communal_taxi_service,"[398850240.0, 398850240.0]",POINT (285287.993 6472823.489),1
4,"POLYGON ((285567.160 6473084.614, 285559.068 6...",4,3913,Kullebacken,307856454,"[9011014436600000, 9011014436600000]","[866, 866]","[9022014024331001, 9022014024331002]","[1, 0]",communal_taxi_service,"[398850240.0, 398850240.0]",POINT (285542.486 6473165.954),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11628,"POLYGON ((506470.633 6547486.923, 506462.542 6...",8468,2511,Hallsberg,297994512,"[9011074160300000, 9011014163200000, 901107416...","[TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG]","[9022014088009002, 9022014088009002, 902201408...","[1, 1, 1, 0, 0, 0, 1, 0, 0]",railway_service,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",POINT (506445.959 6547568.263),1
11629,"POLYGON ((508079.650 6554109.082, 508071.559 6...",8469,3929,Kumla,297884918,"[9011074160400000, 9011074160300000, 901101416...","[TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG]","[9022014088020001, 9022014088020001, 902201408...","[0, 0, 0, 0, 1, 1, 1, 1]",railway_service,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",POINT (508054.976 6554190.422),1
11630,"POLYGON ((511554.639 6570051.618, 511546.548 6...",8470,8715,Örebro södra,297915828,"[9011074160300000, 9011014160200000, 901101416...","[TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG]","[9022014088002001, 9022014088002001, 902201408...","[1, 1, 1, 1, 0, 0, 0, 0]",railway_service,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",POINT (511529.965 6570132.958),1
11631,"POLYGON ((512027.885 6570986.938, 512019.633 6...",8471,8714,Örebro central,297915828,"[9011014163200000, 9011074160300000, 901107416...","[TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, TÅG, ...","[9022014088000003, 9022014088000003, 902201408...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1]",railway_service,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",POINT (512011.302 6571070.304),1


In [15]:
# Step 15: Merge multi-mode buffers with one-mode table
nodes_pt_5_l2_one_mode.loc[:,'stop_areas_name']  = nodes_pt_5_l2_one_mode['stop_name']
nodes_pt_5_l3 = pd.merge(
    nodes_pt_5_l2_one_mode[['stop_areas_name', 'stop_name', 'place_id', 'mode', 'geometry']],
    nodes_pt_5_l2_stop_areas_centroids,
    on=['stop_areas_name', 'stop_name', 'place_id', 'mode', 'geometry'],
    how='outer'
)

C:\Users\monaliza\AppData\Local\Temp\ipykernel_3500\1643996103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_pt_5_l2_one_mode.loc[:,'stop_areas_name']  = nodes_pt_5_l2_one_mode['stop_name']


In [16]:
# Step 16: Merge with nodes_pt_1 to get disaggregated information
nodes_pt_5_l4 = pd.merge(
    nodes_pt_5_l3, nodes_pt_1[['stop_id', 'route_id', 'route_short_name', 'direction_id', 'osmid', 'stop_name', 'place_id', 'mode']],
    on=['stop_name', 'place_id', 'mode'], how='inner'
)

# Step 17 (optional): Display the results
nodes_pt_5_l4.head()

,stop_areas_name,stop_name,place_id,mode,geometry,buffer_id,geometry_buffer,stop_id,route_id,route_short_name,direction_id,osmid
0,7:e Villagatan,7:e Villagatan,307955489,bus_service,POINT (378291.977 6399668.440),NaN,None,9022014082785001,9011014200300000,3,0,87793055.0
1,7:e Villagatan,7:e Villagatan,307955489,bus_service,POINT (378291.977 6399668.440),NaN,None,9022014082785001,9011014200500000,5,1,87793055.0
2,7:e Villagatan,7:e Villagatan,307955489,bus_service,POINT (378291.977 6399668.440),NaN,None,9022014082785001,9011014200800000,8,0,87793055.0
3,7:e Villagatan,7:e Villagatan,307955489,bus_service,POINT (378291.977 6399668.440),NaN,None,9022014082785001,9011014200100000,1,0,87793055.0
4,7:e Villagatan,7:e Villagatan,307955489,bus_service,POINT (378291.977 6399668.440),NaN,None,9022014082785002,9011014200300000,3,1,23859327.0


In [17]:
# Step 18: Create node_id column
nodes_pt_5_l4['node_id'] = nodes_pt_5_l4['stop_areas_name'].astype(str) + \
                            '_' + nodes_pt_5_l4['place_id'].astype(str) + \
                            '_'+ nodes_pt_5_l4['mode'].astype(str)

# Step 18: Aggregate by node_id
agg_func_node={'stop_areas_name':'first',
    'stop_name':list,
    'place_id':'first',
    'mode': 'first',
    'route_id': list,
    'route_short_name': list,
    'stop_id': list,
    'direction_id':list,
    'osmid': list,
    'geometry': 'first'
}

nodes_pt_5_l4 = nodes_pt_5_l4.groupby(['node_id']).agg(agg_func_node).reset_index()

# Step 19 (optional): Display the temporary results
nodes_pt_5_l4.head()

,node_id,stop_areas_name,stop_name,place_id,mode,route_id,route_short_name,stop_id,direction_id,osmid,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,"[7:e Villagatan, 7:e Villagatan, 7:e Villagata...",307955489,bus_service,"[9011014200300000, 9011014200500000, 901101420...","[3, 5, 8, 1, 3, 5, 8, 1]","[9022014082785001, 9022014082785001, 902201408...","[0, 1, 0, 0, 1, 0, 1, 1]","[87793055.0, 87793055.0, 87793055.0, 87793055....",POINT (378291.977 6399668.440)
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,"[AGA Vattenfall, AGA Vattenfall, AGA Vattenfal...",307980814,bus_service,"[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 932, 331, 334, 4]","[9022014015161001, 9022014015161001, 902201401...","[0, 0, 1, 1, 1, 1, 0]","[1021141732.0, 1021141732.0, 1021141732.0, 102...",POINT (313191.955 6442732.480)
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,[Abrahamstorp],307959538,bus_service,[9011014658100000],[581],[9022014046020001],[1],[173516827.0],POINT (371017.481 6448504.455)
3,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,"[Abrahamstorp, Abrahamstorp]",307959538,communal_taxi_service,"[9011014366300000, 9011014366300000]","[663, 663]","[9022014046020001, 9022014046020002]","[1, 0]","[173516827.0, 835571452.0]",POINT (371017.481 6448504.455)
4,Acklinga Solbacken_307927983_bus_service,Acklinga Solbacken,[Acklinga Solbacken],307927983,bus_service,[9011014332600000],[326],[9022014042270002],[1],[1008032531.0],POINT (434327.469 6451981.944)


In [18]:
# Filter and organize the final table
# Step 20: Select desired columns for the final DataFrame
nodes_pt_5 = nodes_pt_5_l4 [['node_id', 'stop_areas_name', 'stop_name', 'place_id',
                             'route_id', 'route_short_name', 'stop_id', 'mode',
                             'direction_id', 'osmid', 'geometry']]

# Step 21: Display the final results
nodes_pt_5.head()

,node_id,stop_areas_name,stop_name,place_id,route_id,route_short_name,stop_id,mode,direction_id,osmid,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,"[7:e Villagatan, 7:e Villagatan, 7:e Villagata...",307955489,"[9011014200300000, 9011014200500000, 901101420...","[3, 5, 8, 1, 3, 5, 8, 1]","[9022014082785001, 9022014082785001, 902201408...",bus_service,"[0, 1, 0, 0, 1, 0, 1, 1]","[87793055.0, 87793055.0, 87793055.0, 87793055....",POINT (378291.977 6399668.440)
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,"[AGA Vattenfall, AGA Vattenfall, AGA Vattenfal...",307980814,"[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 932, 331, 334, 4]","[9022014015161001, 9022014015161001, 902201401...",bus_service,"[0, 0, 1, 1, 1, 1, 0]","[1021141732.0, 1021141732.0, 1021141732.0, 102...",POINT (313191.955 6442732.480)
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,[Abrahamstorp],307959538,[9011014658100000],[581],[9022014046020001],bus_service,[1],[173516827.0],POINT (371017.481 6448504.455)
3,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,"[Abrahamstorp, Abrahamstorp]",307959538,"[9011014366300000, 9011014366300000]","[663, 663]","[9022014046020001, 9022014046020002]",communal_taxi_service,"[1, 0]","[173516827.0, 835571452.0]",POINT (371017.481 6448504.455)
4,Acklinga Solbacken_307927983_bus_service,Acklinga Solbacken,[Acklinga Solbacken],307927983,[9011014332600000],[326],[9022014042270002],bus_service,[1],[1008032531.0],POINT (434327.469 6451981.944)


In [19]:
# Step 22 (optional): create the GeoDataFrame and export to the DB
nodes_pt_5  = gpd.GeoDataFrame(nodes_pt_5, geometry='geometry', crs='3006')
nodes_pt_5.to_postgis('nodes_pt_5', engine, schema='pt_5_odirection_stopareas_bymode', if_exists ='replace')

### Links

In [20]:
#Import data
#Step 0: Import Data
links_pt_1_query = text('SELECT * FROM pt_1_mdirections_mstops_mroutes.links_pt_1_wide')
links_pt_1 = gpd.read_postgis(links_pt_1_query, engine, geom_col='geometry')

In [21]:
# Step 1: Explode nodes table by stop name, clean data
nodes_pt_5_exploded = nodes_pt_5.explode('stop_name').drop_duplicates(['stop_name', 'place_id', 'mode'])

# Step 2: Display the temporary results
nodes_pt_5_exploded.head()

,node_id,stop_areas_name,stop_name,place_id,route_id,route_short_name,stop_id,mode,direction_id,osmid,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,7:e Villagatan,307955489,"[9011014200300000, 9011014200500000, 901101420...","[3, 5, 8, 1, 3, 5, 8, 1]","[9022014082785001, 9022014082785001, 902201408...",bus_service,"[0, 1, 0, 0, 1, 0, 1, 1]","[87793055.0, 87793055.0, 87793055.0, 87793055....",POINT (378291.977 6399668.440)
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,AGA Vattenfall,307980814,"[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 932, 331, 334, 4]","[9022014015161001, 9022014015161001, 902201401...",bus_service,"[0, 0, 1, 1, 1, 1, 0]","[1021141732.0, 1021141732.0, 1021141732.0, 102...",POINT (313191.955 6442732.480)
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,Abrahamstorp,307959538,[9011014658100000],[581],[9022014046020001],bus_service,[1],[173516827.0],POINT (371017.481 6448504.455)
3,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,Abrahamstorp,307959538,"[9011014366300000, 9011014366300000]","[663, 663]","[9022014046020001, 9022014046020002]",communal_taxi_service,"[1, 0]","[173516827.0, 835571452.0]",POINT (371017.481 6448504.455)
4,Acklinga Solbacken_307927983_bus_service,Acklinga Solbacken,Acklinga Solbacken,307927983,[9011014332600000],[326],[9022014042270002],bus_service,[1],[1008032531.0],POINT (434327.469 6451981.944)


In [22]:
# Preprocess the Data
# Step 1: Merge nodes and links in order to get the right combination of data
links_pt_5_source_merge = pd.merge(
    nodes_pt_5_exploded[['node_id', 'stop_areas_name', 'stop_name', 'place_id',
                         'mode', 'direction_id', 'geometry']],
    links_pt_1 [['route_id', 'route_short_name', 'stop_id_source', 'stop_id_target',
                 'stop_name_source', 'stop_name_target', 'time_distance', 'place_id_source',
                 'place_id_target', 'mode'] + [col for col in links_pt_1.columns if col.startswith('freq_')]],
    left_on=['mode', 'stop_name', 'place_id'],
    right_on=['mode','stop_name_source','place_id_source'],
    how='inner'
)

links_pt_5_target_merge  = pd.merge(
    nodes_pt_5_exploded[['node_id', 'stop_areas_name', 'stop_name', 'place_id',
                         'mode', 'direction_id', 'geometry']],
    links_pt_5_source_merge[['node_id', 'stop_areas_name', 'stop_name_source', 'stop_name_target',
                             'place_id_source', 'place_id_target', 'stop_id_source',
                             'stop_id_target', 'time_distance', 'route_id', 'route_short_name',
                             'mode', 'geometry'] + [col for col in links_pt_5_source_merge.columns if col.startswith('freq_')]],
    left_on=['mode', 'stop_name', 'place_id'],
    right_on=['mode', 'stop_name_target', 'place_id_target'],
    how='inner'
).rename(columns={
    'node_id_x':'node_id_target',
    'node_id_y':'node_id_source',
    'stop_areas_name_x': 'stop_areas_name_target',
    'stop_areas_name_y': 'stop_areas_name_source',
    'geometry_x':'geometry_target',
    'geometry_y':'geometry_source'})

# Step 2: Select specific columns and remove duplicates
links_pt_5_l1 = links_pt_5_target_merge [['node_id_source', 'node_id_target', 'time_distance',
                                          'stop_areas_name_source', 'stop_areas_name_target',
                                          'stop_name_source', 'stop_name_target', 'place_id_source',
                                          'place_id_target', 'stop_id_source', 'stop_id_target',
                                          'route_id', 'route_short_name', 'mode', 'direction_id',
                                          'geometry_source','geometry_target']+ [col for col in links_pt_5_target_merge.columns if col.startswith('freq_')]]

# Step 2 (optional): Display the results
links_pt_5_l1.head()

,node_id_source,node_id_target,time_distance,stop_areas_name_source,stop_areas_name_target,stop_name_source,stop_name_target,place_id_source,place_id_target,stop_id_source,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,129.0,Bäckängsskolan,7:e Villagatan,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,...,0.0,0.0,0.0,2842.105263,1800.000000,2571.428571,1800.000000,2454.545455,4909.090909,0.0
1,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,114.0,Bäckängsskolan,7:e Villagatan,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,...,0.0,0.0,0.0,9000.000000,9671.641791,9000.000000,9000.000000,9671.641791,54000.000000,0.0
2,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,117.0,Bäckängsskolan,7:e Villagatan,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,...,0.0,0.0,0.0,8415.584416,5102.362205,8415.584416,4729.927007,6750.000000,9000.000000,0.0
3,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,117.0,Bäckängsskolan,7:e Villagatan,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,...,0.0,0.0,0.0,6000.000000,3410.526316,4984.615385,3100.478469,5634.782609,9000.000000,0.0
4,Södra Älvsborgs sjukhus_307955489_bus_service,7:e Villagatan_307955489_bus_service,98.0,Södra Älvsborgs sjukhus,7:e Villagatan,Södra Älvsborgs sjukhus,7:e Villagatan,307955489,307955489,9022014082726001,...,0.0,0.0,0.0,2781.115880,1825.352113,2623.481781,1800.000000,2347.826087,6000.000000,0.0


In [23]:
# Create the links of the one direction model
# Step 3: Aggregate Data
links_pt_5_l1['geometry'] = links_pt_5_l1.apply(lambda row: LineString([Point(row['geometry_source']), Point(row['geometry_target'])]), axis=1)
links_pt_5_l1['symmetric_pairs'] = links_pt_5_l1[['node_id_source', 'node_id_target']].apply(lambda x: '-'.join(sorted(x)), axis=1)

# Step 4: Define aggregation functions for various columns
agg_func_link = {
    'node_id_source': 'first',
    'node_id_target': 'first',
    'stop_areas_name_source': 'first',
    'stop_areas_name_target': 'first',
    'mode':'first',
    'route_id': list,
    'route_short_name': list,
    'stop_id_source': 'first',
    'stop_id_target': 'first',
    'stop_name_source': 'first',
    'stop_name_target': 'first',
    'place_id_source': 'first',
    'place_id_target': 'first',
    'time_distance': 'min',
    'direction_id': 'first',
    'geometry': 'first'
}

freq_cols = [col for col in links_pt_5_l1.columns if col.startswith('freq_')]
agg_func_link.update({col: 'min' for col in freq_cols})

links_pt_5_l2 = links_pt_5_l1.groupby(['symmetric_pairs']).agg(agg_func_link).reset_index()

# Step 6 (optional): Display the results
links_pt_5_l2.head()

C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,symmetric_pairs,node_id_source,node_id_target,stop_areas_name_source,stop_areas_name_target,mode,route_id,route_short_name,stop_id_source,stop_id_target,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_bus_service-Bäckängss...,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,Bäckängsskolan,7:e Villagatan,bus_service,"[9011014200100000, 9011014200300000, 901101420...","[1, 3, 5, 8, 1, 3, 5, 8]",9022014082676002,9022014082785002,...,0.0,0.0,0.0,2842.105263,1800.000000,2571.428571,1800.0,2347.826087,0.0,0.0
1,7:e Villagatan_307955489_bus_service-Södra Älv...,Södra Älvsborgs sjukhus_307955489_bus_service,7:e Villagatan_307955489_bus_service,Södra Älvsborgs sjukhus,7:e Villagatan,bus_service,"[9011014200100000, 9011014200300000, 901101420...","[1, 3, 5, 8, 1, 3, 5, 8]",9022014082726001,9022014082785001,...,0.0,0.0,0.0,2757.446809,1825.352113,2623.481781,1800.0,2347.826087,0.0,0.0
2,AGA Vattenfall_307980814_bus_service-Borealis ...,Borealis kracker_307980814_bus_service,AGA Vattenfall_307980814_bus_service,Borealis kracker,AGA Vattenfall,bus_service,"[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 331, 334, 4, 932]",9022014015013002,9022014015161002,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
3,AGA Vattenfall_307980814_bus_service-Jordhamma...,Jordhammar_307980814_bus_service,AGA Vattenfall_307980814_bus_service,Jordhammar,AGA Vattenfall,bus_service,"[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 932, 331, 334, 4]",9022014015110001,9022014015161001,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
4,Abrahamstorp_307959538_bus_service-Badet_30795...,Badet_307959538_bus_service,Abrahamstorp_307959538_bus_service,Badet,Abrahamstorp,bus_service,[9011014658100000],[581],9022014046021001,9022014046020001,...,0.0,0.0,0.0,54000.000000,27000.000000,27000.000000,27000.0,27000.000000,0.0,0.0


In [24]:
# Filter and organize the final table
# Step 7: Rename columns for clarity
links_pt_5_l2.rename(
    columns={'node_id_source': 'source', 'node_id_target': 'target'},
    inplace=True
)

# Step 8: Select desired columns for the final DataFrame
links_pt_5 = links_pt_5_l2[['source', 'target', 'time_distance', 'route_id', 'stop_name_source',
                            'place_id_source', 'stop_name_target', 'place_id_target',
                            'stop_id_source', 'stop_id_target', 'route_short_name', 'mode',
                            'direction_id','geometry']
                           + [col for col in links_pt_5_l2.columns if col.startswith('freq_')]].drop_duplicates(subset=['source', 'target'])

# Step 9: Filter out rows where 'source' is equal to 'target'
links_pt_5 = links_pt_5[links_pt_5['source'] != links_pt_5['target']]

# Step 10 : Display the final results
links_pt_5.head()

,source,target,time_distance,route_id,stop_name_source,place_id_source,stop_name_target,place_id_target,stop_id_source,stop_id_target,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,109.0,"[9011014200100000, 9011014200300000, 901101420...",Bäckängsskolan,307955489,7:e Villagatan,307955489,9022014082676002,9022014082785002,...,0.0,0.0,0.0,2842.105263,1800.000000,2571.428571,1800.0,2347.826087,0.0,0.0
1,Södra Älvsborgs sjukhus_307955489_bus_service,7:e Villagatan_307955489_bus_service,86.0,"[9011014200100000, 9011014200300000, 901101420...",Södra Älvsborgs sjukhus,307955489,7:e Villagatan,307955489,9022014082726001,9022014082785001,...,0.0,0.0,0.0,2757.446809,1825.352113,2623.481781,1800.0,2347.826087,0.0,0.0
2,Borealis kracker_307980814_bus_service,AGA Vattenfall_307980814_bus_service,27.0,"[9011014633100000, 9011014633400000, 901101463...",Borealis kracker,307980814,AGA Vattenfall,307980814,9022014015013002,9022014015161002,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
3,Jordhammar_307980814_bus_service,AGA Vattenfall_307980814_bus_service,97.0,"[9011014633100000, 9011014633400000, 901101463...",Jordhammar,307980814,AGA Vattenfall,307980814,9022014015110001,9022014015161001,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
4,Badet_307959538_bus_service,Abrahamstorp_307959538_bus_service,82.0,[9011014658100000],Badet,307959538,Abrahamstorp,307959538,9022014046021001,9022014046020001,...,0.0,0.0,0.0,54000.000000,27000.000000,27000.000000,27000.0,27000.000000,0.0,0.0


In [25]:
# Step 11 (optional): Set geometry and export to the DB
links_pt_5 = gpd.GeoDataFrame(links_pt_5, geometry='geometry', crs='3006')
links_pt_5.to_postgis('links_pt_5', engine, schema='pt_5_odirection_stopareas_bymode', if_exists ='replace')

### Transfers

#### Same stop

In [26]:
# Preprocess the Data
# Step 1: Create a copy of the nodes table
nodes_transfer_same_stop = nodes_pt_5[['node_id', 'stop_areas_name', 'place_id', 'geometry']].copy()

#Step 2 (optional): display the results
nodes_transfer_same_stop.head()

,node_id,stop_areas_name,place_id,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480)
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371017.481 6448504.455)
3,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,307959538,POINT (371017.481 6448504.455)
4,Acklinga Solbacken_307927983_bus_service,Acklinga Solbacken,307927983,POINT (434327.469 6451981.944)


In [27]:
# Step 3: Merge the stops in order to create the transfers
# Source
stop_routes_source_merge = nodes_transfer_same_stop.rename(columns = {'node_id': 'source',
                                                                      'stop_areas_name': 'target'}).drop_duplicates()

# Target
stop_routes_target_merge = pd.merge(nodes_transfer_same_stop,
                                    nodes_pt_5[['node_id']],
                                    on='node_id',
                                    how='inner').rename(columns = {'stop_areas_name':'source',
                                                                   'node_id':'target'}).drop_duplicates()

# Step 4: Create the transfers same stop table
transfers_same_stop = pd.merge(stop_routes_source_merge,
                               stop_routes_target_merge,
                               how='outer')

#Step 5 (optional): Display the results
transfers_same_stop.head()

,source,target,place_id,geometry
0,7:e Villagatan,7:e Villagatan_307955489_bus_service,307955489,POINT (378291.977 6399668.440)
1,7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
2,AGA Vattenfall,AGA Vattenfall_307980814_bus_service,307980814,POINT (313191.955 6442732.480)
3,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480)
4,Abrahamstorp,Abrahamstorp_307959538_bus_service,307959538,POINT (371017.481 6448504.455)


In [28]:
# Step 6: Merge the table with the frequency of the links table
transfers_same_stop = pd.merge(transfers_same_stop,
                               links_pt_5[['target'] + [col for col in links_pt_5.columns if col.startswith('freq_')]],
                               on='target',
                               how='outer').drop_duplicates(subset=('source',
                                                                    'target'))
transfers_same_stop = transfers_same_stop [['source',
                                            'target',
                                            'geometry'] + [col for col in transfers_same_stop.columns if col.startswith('freq_')]]

#Step 7 (optional): Display the results
transfers_same_stop.head()

,source,target,geometry,freq_monday_morning,freq_tuesday_morning,freq_wednesday_morning,freq_thursday_morning,freq_friday_morning,freq_saturday_morning,freq_sunday_morning,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7:e Villagatan,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440),568.421053,568.421053,568.421053,568.421053,568.421053,635.294118,635.294118,...,0.0,0.0,0.0,2842.105263,1800.0,2571.428571,1800.0,2347.826087,0.0,0.0
3,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,POINT (313191.955 6442732.480),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGA Vattenfall,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
6,Abrahamstorp_307959538_bus_service,Abrahamstorp,POINT (371017.481 6448504.455),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Step 8: Replace 0 with NaN and vice versa in frequency columns
freq_columns = [col for col in transfers_same_stop.columns if col.startswith('freq')]
transfers_same_stop [freq_columns] = transfers_same_stop[freq_columns].replace({0: np.nan, np.nan: 0})

#Step 9 (optional): Display the results
transfers_same_stop.head()

,source,target,geometry,freq_monday_morning,freq_tuesday_morning,freq_wednesday_morning,freq_thursday_morning,freq_friday_morning,freq_saturday_morning,freq_sunday_morning,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,POINT (378291.977 6399668.440),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
1,7:e Villagatan,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440),568.421053,568.421053,568.421053,568.421053,568.421053,635.294118,635.294118,...,NaN,NaN,NaN,2842.105263,1800.0,2571.428571,1800.0,2347.826087,NaN,NaN
3,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,POINT (313191.955 6442732.480),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,AGA Vattenfall,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Abrahamstorp_307959538_bus_service,Abrahamstorp,POINT (371017.481 6448504.455),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0


In [30]:
# Step 10 (optional): Set geometry and export to the DB
transfers_same_stop = gpd.GeoDataFrame(transfers_same_stop, geometry='geometry',  crs='3006')
transfers_same_stop.to_postgis('transfers_same_stop_pt_5', engine, schema='pt_5_odirection_stopareas_bymode', if_exists ='replace')

#### Different stops

In [31]:
#Preprocess the Data
# Step 1: Create a copy of the nodes table
nodes_transfer_diff_stop = nodes_pt_5[['stop_areas_name', 'place_id', 'node_id','geometry']]

# Step 2 (optional): Display the results
nodes_transfer_diff_stop.head()

,stop_areas_name,place_id,node_id,geometry
0,7:e Villagatan,307955489,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440)
1,AGA Vattenfall,307980814,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480)
2,Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455)
3,Abrahamstorp,307959538,Abrahamstorp_307959538_communal_taxi_service,POINT (371017.481 6448504.455)
4,Acklinga Solbacken,307927983,Acklinga Solbacken_307927983_bus_service,POINT (434327.469 6451981.944)


In [32]:
# Create the buffers
# Step 3: Create a buffer column and set it as geometry
nodes_transfer_diff_stop['buffer'] = nodes_transfer_diff_stop['geometry'].buffer(85)
nodes_transfer_diff_stop = nodes_transfer_diff_stop.set_geometry('buffer',
                                                                 crs='EPSG:3006')

# Step 4: found the intersections among the buffers
nodes_transfer_diff_stop = gpd.overlay(nodes_transfer_diff_stop,
                                       nodes_transfer_diff_stop,
                                       how='intersection').rename(columns={'node_id_1':'source',
                                                                           'node_id_2':'target',
                                                                           'stop_areas_name_1':'stop_areas_name_source',
                                                                           'stop_areas_name_2':'stop_areas_name_target',
                                                                           'place_id_1':'place_id_source',
                                                                           'place_id_2':'place_id_target',
                                                                           'geometry_1': 'geometry_source',
                                                                           'geometry_2': 'geometry_target'})

# Step 5 (optional): Display the results
nodes_transfer_diff_stop.head()

C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,stop_areas_name_source,place_id_source,source,geometry_source,stop_areas_name_target,place_id_target,target,geometry_target,geometry
0,7:e Villagatan,307955489,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440),7:e Villagatan,307955489,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440),"POLYGON ((378376.568 6399660.109, 378375.344 6..."
1,AGA Vattenfall,307980814,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480),AGA Vattenfall,307980814,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480),"POLYGON ((313276.546 6442724.148, 313275.322 6..."
2,Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455),Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455),"POLYGON ((371102.072 6448496.124, 371100.848 6..."
3,Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455),Abrahamstorp,307959538,Abrahamstorp_307959538_communal_taxi_service,POINT (371017.481 6448504.455),"POLYGON ((371102.072 6448496.124, 371100.848 6..."
4,Abrahamstorp,307959538,Abrahamstorp_307959538_communal_taxi_service,POINT (371017.481 6448504.455),Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455),"POLYGON ((371102.072 6448496.124, 371100.848 6..."


In [33]:
#Create the geometry
# Step 6: Create linstrings between the intersections
nodes_transfer_diff_stop['geometry'] = nodes_transfer_diff_stop.apply(
    lambda row: LineString([[row.geometry_source.x, row.geometry_source.y],
                            [row.geometry_target.x, row.geometry_target.y]])
    if row.geometry_target is not None else None,
    axis=1)

# Step 7: Set the length of the linestring
nodes_transfer_diff_stop['length'] = nodes_transfer_diff_stop['geometry'].length
nodes_transfer_diff_stop['time_distance'] = nodes_transfer_diff_stop['length'] / 1.2

# Step 8 (optional): Display the results
nodes_transfer_diff_stop.head()

,stop_areas_name_source,place_id_source,source,geometry_source,stop_areas_name_target,place_id_target,target,geometry_target,geometry,length,time_distance
0,7:e Villagatan,307955489,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440),7:e Villagatan,307955489,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440),"LINESTRING (378291.977 6399668.440, 378291.977...",0.0,0.0
1,AGA Vattenfall,307980814,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480),AGA Vattenfall,307980814,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480),"LINESTRING (313191.955 6442732.480, 313191.955...",0.0,0.0
2,Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455),Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455),"LINESTRING (371017.481 6448504.455, 371017.481...",0.0,0.0
3,Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455),Abrahamstorp,307959538,Abrahamstorp_307959538_communal_taxi_service,POINT (371017.481 6448504.455),"LINESTRING (371017.481 6448504.455, 371017.481...",0.0,0.0
4,Abrahamstorp,307959538,Abrahamstorp_307959538_communal_taxi_service,POINT (371017.481 6448504.455),Abrahamstorp,307959538,Abrahamstorp_307959538_bus_service,POINT (371017.481 6448504.455),"LINESTRING (371017.481 6448504.455, 371017.481...",0.0,0.0


In [34]:
# Filter and organize the tables
# Step 9: Filter the transfers between different stops
transfers_diff_stop = nodes_transfer_diff_stop[nodes_transfer_diff_stop['length'] > 0]

# Step 10: Drop the duplicates of symmetric pairs
transfers_diff_stop ['symmetric_pairs'] = transfers_diff_stop[['source',
                                                               'target']].apply(lambda x: '-'.join(sorted(x)),
                                                                                axis=1)
transfers_diff_stop = transfers_diff_stop.drop_duplicates(subset = ['symmetric_pairs'])

# Step 11: Filter the necessary columns
transfers_diff_stop = transfers_diff_stop [['stop_areas_name_source',
                                            'stop_areas_name_target',
                                            'geometry',
                                            'length',
                                            'time_distance']].rename(columns={'stop_areas_name_source':'source',
                                                                              'stop_areas_name_target':'target'}).drop_duplicates()
# Step 12 : Display the final results
transfers_diff_stop.head()

C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,source,target,geometry,length,time_distance
64,Alekärrsgatan,Dickson Södra,"LINESTRING (321849.955 6403991.951, 321732.984...",120.391991,100.326659
109,Allégatan,Södra torget,"LINESTRING (377468.217 6399358.673, 377421.472...",97.224317,81.020264
110,Allégatan,Åsbogatan,"LINESTRING (377468.217 6399358.673, 377515.487...",48.835419,40.696182
160,Alvareds bro,Storegården,"LINESTRING (414333.450 6432792.434, 414431.478...",100.353921,83.628267
249,Anneholmsgatan,Prytzgatan,"LINESTRING (321473.976 6398939.953, 321578.993...",124.548843,103.790702


In [35]:
#export to the db
transfers_diff_stop = gpd.GeoDataFrame(transfers_diff_stop, geometry='geometry', crs='3006')
transfers_diff_stop.to_postgis('pt_5_odirection_stopareas_bymode', engine, schema='pt_5_odirection_stopareas_bymode', if_exists ='replace')